In [ ]:
import math
import numpy as np
import torch
import torch.utils.checkpoint
import torchvision
import matplotlib.pyplot as plt

from torchvision import transforms
from tqdm.auto import tqdm

from rectified_flow.rectified_flow import RectifiedFlow
from rectified_flow.utils import match_dim_with_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
batch_size = 512

transform_list = [
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
]
train_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.Compose(transform_list)
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,          
    pin_memory=True,
    persistent_workers=False,
)

batch = next(iter(train_dataloader))
print(batch[0].shape)  # torch.Size([256, 1, 28, 28])

In [ ]:
model_type = "unet"
from rectified_flow.models.enhanced_mlp import VarMLP
from rectified_flow.models.utils import EMAModel
from rectified_flow.models.unet import SongUNet, SongUNetConfig

flow_model = SongUNet.from_pretrained("/scratch/10992/liaorunlong93/random_flow_toys/checkpoints/flow_mnist_unet_unconditional", use_ema=True).to(device)

In [ ]:
from rectified_flow.samplers import EulerSampler, SDESampler
from rectified_flow.utils import plot_cifar_results

rf = RectifiedFlow(
    data_shape=(1, 28, 28),
    velocity_field=flow_model,
    device=device,
)

x_0_src = torch.randn(50, 1, 28, 28, device=device)

sampler = EulerSampler(rectified_flow=rf, num_steps=200)
rf_samples = sampler.sample_loop(x_0=x_0_src).trajectories[-1]

sde_sampler = SDESampler(rectified_flow=rf, num_steps=100, noise_decay_rate=0.0, noise_scale=50.0)
rf_sde_samples = sde_sampler.sample_loop(x_0=x_0_src).trajectories[-1]

plot_cifar_results(rf_samples)
plot_cifar_results(rf_sde_samples)

## Implement divergence estimation and verify correctness

### Exact Jacobian calculation with autograd

In [ ]:
from tqdm import tqdm
from torch.autograd.functional import jacobian

def divergence_exact_manual(v_func, x_t, t):
    device, dtype = x_t.device, x_t.dtype
    B = x_t.shape[0]
    div = torch.empty(B, device=device, dtype=dtype)

    for b in tqdm(range(B), desc="Computing exact divergence"):
        xb = x_t[b:b+1].detach().requires_grad_(True)
        tb = t[b:b+1]

        def f(inp):
            return v_func(inp, tb)

        J = jacobian(f, xb, vectorize=False, create_graph=False)
        D = xb.numel()
        div[b] = J.reshape(D, D).diagonal().sum().to(dtype)
        del J
        
    return div

### Divergence Estimation via the Hutchinson Trick

For a vector field $v(x,t)$ with Jacobian $J(x) = \dfrac{\partial v}{\partial x}$, the divergence is
$$
\operatorname{div} v(x,t) = \operatorname{tr}\, J(x).
$$
Computing $\operatorname{tr} J$ via an explicit Jacobian is expensive. The Hutchinson identity gives an unbiased estimator:
$$
\operatorname{tr} J = \mathbb{E}_{\varepsilon}\!\left[\varepsilon^\top J \varepsilon\right],
$$
where $\varepsilon$ can be Rademacher ($\pm1$ w.p. 1/2) or standard Gaussian. A Monte Carlo estimate is
$$
\widehat{\operatorname{div}}(x,t) = \frac{1}{M}\sum_{i=1}^{M} \varepsilon_i^\top (J\,\varepsilon_i),
$$
which only requires Jacobian–vector products (JVP) or vector–Jacobian products (VJP), avoiding materializing the full Jacobian.

**Implementation notes**
- `eps_dist="rademacher"` often yields slightly lower variance; `"normal"` works too.
- Use forward-mode `jvp` to get $J\varepsilon$, or a single backward (VJP) to get $J^\top \varepsilon$ and then dot with $\varepsilon$.
- Compute per-sample inner products in the batch and average across $M$ probes; `M=1–4` is typically sufficient.
- When using fp16/bf16, accumulate in fp32 for numerical stability.


In [ ]:
from torch.func import jvp as jvp_func

def divergence_hutchinson(
    v_func,
    x_t: torch.Tensor,
    t: torch.Tensor,
    n_samples: int = 20,
    eps_dist: str = "rademacher",
    method: str = "vjp",
) -> torch.Tensor:
    device, dtype = x_t.device, x_t.dtype
    B = x_t.shape[0]
    D = x_t.numel() // B

    x_flat = x_t.detach().view(B, D)

    acc_dtype = torch.float32 if dtype in (torch.float16, torch.bfloat16) else dtype
    acc = torch.zeros(B, device=device, dtype=acc_dtype)

    def _sample_eps_flat():
        if eps_dist == "rademacher":
            r = torch.randint(0, 2, (B, D), device=device)
            eps = (r * 2 - 1).to(acc_dtype)
        elif eps_dist == "normal":
            eps = torch.randn((B, D), device=device, dtype=acc_dtype)
        else:
            raise ValueError("eps_dist must be 'rademacher' or 'normal'")
        return eps.to(dtype)

    if method == "jvp":
        def f_flat(inp_flat):
            x4 = inp_flat.view(B, *x_t.shape[1:])
            y4 = v_func(x4, t)
            return y4.view(B, D)

        for _ in range(n_samples):
            eps_flat = _sample_eps_flat()
            _, jvp_out = jvp_func(f_flat, (x_flat,), (eps_flat,), strict=False)  # <-- fix
            acc += (eps_flat * jvp_out).sum(dim=1).to(acc_dtype)

        return (acc / n_samples).to(dtype)

    elif method == "vjp":
        for _ in range(n_samples):
            eps_flat = _sample_eps_flat()
            eps = eps_flat.view_as(x_t)
            x_req = x_t.detach().requires_grad_(True)
            v = v_func(x_req, t)
            s_per_sample = (v * eps).view(B, -1).sum(dim=1)
            gx = torch.autograd.grad(
                s_per_sample.sum(), x_req, create_graph=False, retain_graph=False
            )[0]
            acc += (gx.detach() * eps).view(B, -1).sum(dim=1).to(acc_dtype)
            del x_req, v, gx, eps, eps_flat, s_per_sample

        return (acc / n_samples).to(dtype)

    else:
        raise ValueError("method must be 'jvp' or 'vjp'")

In [ ]:
x_1 = batch[0].to(device)[233:234]
x_1 = x_1.repeat_interleave(6, dim=0)
t = torch.linspace(0.95, 1.0, steps=x_1.shape[0], device=device)
x_t = t[:, None, None, None] * x_1 + (1 - t)[:, None, None, None] * torch.randn_like(x_1)

div_exact = divergence_exact_manual(flow_model, x_t, t)
div_approx = divergence_hutchinson(flow_model, x_t, t, n_samples=20, method="vjp")

print("Divergence shape:", div_exact.shape)
print("div exact:", div_exact)
print("div approx:", div_approx)
print("Max absolute error:", (div_exact - div_approx).abs().max().item())

## log prob estimation with Simultaneous change of variable


$$
\log \rho_t(x^{\mathrm{eval}}) 
= 
\log \rho_0(z_0^{\mathrm{eval}}) 
- 
\int_0^1 \nabla \cdot v_t(z_t^{\mathrm{eval}}) \, \mathrm dt,
$$

where $ \{ z_t^{\mathrm{eval}} \} $ is the solution of 

$$
\dot{z}_t^{\mathrm{eval}} = v_t(z_t^{\mathrm{eval}}), 
\quad \text{with} \quad 
z_1^{\mathrm{eval}} = x^{\mathrm{eval}}.
$$



In [ ]:
n_samples = 30

x_eval = batch[0].to(device)[:n_samples]
x_eval = torch.cat([
        rf_samples[:n_samples],     # ODE inv
        rf_sde_samples[:n_samples], # SDE inv
        x_eval,              # Test set inv
    ], dim=0)
alpha = 0.01
x_eval = (1. - alpha) * x_eval + alpha * torch.randn_like(x_eval)
# x_eval = x_1
N = 1000
plot_cifar_results(x_eval)

# Generate an ODE trajectory for evaluation
time_grid = torch.linspace(0., 1. - alpha, N, device=device)
inv_rf = RectifiedFlow(
    data_shape=(1, 28, 28),
    velocity_field=lambda x, t: -flow_model(x, 1. - t),
    device=device,
)
sampler_inv = EulerSampler(rectified_flow=inv_rf, time_grid=time_grid)
x_0_inv = sampler_inv.sample_loop(x_0=x_eval).trajectories[-1]
x_0_inv = torch.cat((x_0_src[:n_samples], x_0_inv), dim=0) # [ODE vanilla, ODE inv, SDE inv, Test set inv]
plot_cifar_results(x_0_inv)

# Now generate samples from x_0_inv back to x_eval, to see how well we reconstruct
rf = RectifiedFlow(
    data_shape=(1, 28, 28),
    velocity_field=flow_model,
    device=device,
)
sampler = EulerSampler(rectified_flow=rf, num_samples=50, num_steps=N)
x_1 = sampler.sample_loop(x_0=x_0_inv).trajectories[-1]
plot_cifar_results(x_1)
plot_cifar_results(torch.abs(x_1[n_samples:] - x_eval))

In [ ]:
import seaborn as sns
import scipy.stats as stats

x = x_0_inv.view(-1).cpu().numpy()
sns.histplot(x, kde=True, stat="density")
mu, sigma = x.mean(), x.std()
xs = np.linspace(x.min(), x.max(), 200)
plt.plot(xs, stats.norm.pdf(xs, mu, sigma), 'r--', label='Normal fit')
plt.legend()
plt.show()

In [ ]:
def standard_normal_logprob(z):
    B = z.shape[0]
    zf = z.view(B, -1)
    D = zf.shape[1]
    return -0.5 * (zf.pow(2).sum(dim=1) + D * math.log(2 * math.pi))

def estimate_logprob_from_inverse(
    x_0_inv: torch.Tensor,
    rf,                      
    flow_model,
    N: int = 300,
    alpha: float = 0.0,       
    sample_stride: int = 5,
):
    device = x_0_inv.device
    B = x_0_inv.shape[0]

    t1 = max(0.0, 1.0 - alpha)
    time_grid = torch.linspace(0., t1, N, device=device)

    sampler_fwd = EulerSampler(rectified_flow=rf, time_grid=time_grid)
    traj = sampler_fwd.sample_loop(x_0=x_0_inv).trajectories
    states = torch.stack(traj, dim=0) if isinstance(traj, list) else traj     # (T, B, C, H, W)
    T = states.shape[0]

    idx = torch.arange(0, T, sample_stride, device=device)
    if idx[-1].item() != T - 1:
        idx = torch.cat([idx, torch.tensor([T - 1], device=device)])
    t_used = time_grid.index_select(0, idx)               # (Ts,)
    x_used = states.index_select(0, idx)                  # (Ts, B, C, H, W)

    div_values = []
    for k in tqdm(range(t_used.shape[0]), desc="Computing divergences"):
        tk = t_used[k]
        xk = x_used[k]                                     # (B, C, H, W)
        tb = torch.full((B,), tk.item(), device=device)    # (B,)
        div_k = divergence_hutchinson(flow_model, xk, tb, n_samples=30)  # (B,)
        div_values.append(div_k)
        print(f"Time {tk.item():.5f} divergence mean: {div_k.mean().item():.4f}")
    divs = torch.stack(div_values, dim=0)                  # (Ts, B)

    int_div = torch.trapz(divs, t_used, dim=0)            # (B,)
    log_rho0 = standard_normal_logprob(x_0_inv)           # (B,)
    log_rho_eval = log_rho0 - int_div                     # (B,)

    details = {
        "time_grid_full": time_grid,   # (T,)
        "time_grid_used": t_used,      # (Ts,)
        "states_used": x_used,         # (Ts, B, C, H, W)
        "divergences": divs,           # (Ts, B)
        "int_divergence": int_div,     # (B,)
        "log_rho0": log_rho0,          # (B,)
    }
    return log_rho_eval, details

log_rho_eval, details = estimate_logprob_from_inverse(
    x_0_inv=x_0_inv,
    rf=rf,
    flow_model=flow_model,
    N=1000,
    alpha=0.0,
    sample_stride=20,
)

In [ ]:
# [ODE vanilla, ODE inv, SDE inv, Test set inv]

print("ODE_vanilla logp mean",log_rho_eval[:n_samples].mean())

print("ODE_inv logp mean",log_rho_eval[n_samples:2*n_samples].mean())

print("SDE_inv logp mean",log_rho_eval[2*n_samples:3*n_samples].mean())

print("Test_set logp mean",log_rho_eval[3*n_samples:4*n_samples].mean())

In [ ]:
def bpd_from_logrho(log_rho, D, b=8, a=1.0):
    # logrho: per-sample log ρ1(x) in nats (model coordinate)
    # D: H*W*C,  a: per-dim linear scale from y∈[0,1] to x (e.g., a=2 for [-1,1])
    import math
    return -log_rho / (D * math.log(2.0)) + b - math.log2(a)

bpd_eval = bpd_from_logrho(log_rho_eval, D=28*28*1, b=8, a=1.0)
print("BPD Eval mean:", bpd_eval.mean().item())

In [ ]:
details["log_rho0"]

In [ ]:
@torch.no_grad()
def plot_images_with_logrho(
    images: torch.Tensor,
    log_rho: torch.Tensor,
    nrow: int = 10,
    title: str | None = None,
    digits: int = 2,
    dpi: int = 300,
    fontsize: int = 8,
    normalize_from_neg1_pos1: bool = True,
    save_path: str | None = None,
    group_titles: list[str] | None = None,
    group_size: int | None = None,
    show_group_mean: bool = False,
):
    import math
    import torch
    import matplotlib.pyplot as plt
    from matplotlib import gridspec

    assert images.ndim == 4, "images should be (B, C, H, W)"
    B = images.shape[0]
    cols = min(nrow, B)

    imgs = images.detach().cpu().clone()
    if normalize_from_neg1_pos1:
        imgs = (imgs * 0.5 + 0.5).clamp(0, 1)

    scores = log_rho.detach().cpu().view(-1)
    assert scores.numel() == B, "log_rho length must match number of images"

    # 分段
    groups = []
    if group_titles is not None and group_size is not None:
        total_needed = group_size * len(group_titles)
        assert total_needed <= B, "group_size * len(group_titles) exceeds number of images"
        start = 0
        for name in group_titles:
            end = start + group_size
            groups.append((name, start, end))
            start = end
    else:
        groups.append(("All", 0, B))

    rows_per_group = [math.ceil((e - s) / cols) for _, s, e in groups]

    col_width  = 1.35
    row_height = 1.30
    title_frac = 0.10     
    group_hspace = 0.16   
    cell_hspace  = 0.08   
    cell_wspace  = 0.03   

    fig_w = cols * col_width
    fig_h = sum(r * row_height + title_frac * row_height for r in rows_per_group)

    fig = plt.figure(figsize=(fig_w, fig_h), dpi=dpi)
    outer = gridspec.GridSpec(
        nrows=len(groups), ncols=1,
        height_ratios=[(title_frac + r) for r in rows_per_group],
        hspace=group_hspace,
    )

    for gi, (gname, s, e) in enumerate(groups):
        r = rows_per_group[gi]
        sub = outer[gi].subgridspec(
            r + 1, cols,
            height_ratios=[title_frac] + [1]*r,
            wspace=cell_wspace, hspace=cell_hspace
        )

        ax_title = fig.add_subplot(sub[0, :])
        ax_title.axis("off")
        if show_group_mean:
            mean_val = scores[s:e].mean().item()
            txt = f"{gname}  (mean log ρ = {mean_val:.{digits}f})"
        else:
            txt = gname
        ax_title.text(0.0, 0.5, txt, fontsize=fontsize+1, va="center", ha="left")

        for k in range(r * cols):
            ax = fig.add_subplot(sub[1 + k // cols, k % cols])
            idx = s + k
            if idx < e:
                img = imgs[idx]
                if img.shape[0] == 1:
                    ax.imshow(img.squeeze(0), cmap="gray")
                else:
                    ax.imshow(img.permute(1, 2, 0).numpy())
                ax.set_xticks([]); ax.set_yticks([])
                for sp in ax.spines.values():
                    sp.set_visible(False)

                # 把分数放到图内左下角，避免 xlabel 留白
                label = f"{scores[idx].item():.{digits}f}"
                ax.text(
                    0.02, 0.02, label,
                    transform=ax.transAxes, ha="left", va="bottom",
                    fontsize=fontsize, color="white",
                    bbox=dict(facecolor="black", alpha=0.35, pad=1.0, edgecolor="none"),
                )
            else:
                ax.axis("off")

    if title is not None:
        fig.suptitle(title, fontsize=12, y=0.995)

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight", pad_inches=0.05)
    plt.show()


plot_images_with_logrho(
    images=details["states_used"][-1],
    log_rho=log_rho_eval[:],
    nrow=10,
    title="Samples with log ρ",
    digits=2,
    dpi=300,
    fontsize=8,
    group_titles=["ODE vanilla", "ODE inv", "SDE inv", "Test set inv"],
    group_size=n_samples,
    show_group_mean=True,
)

## Log Likelihood Estimation from KL Divergence 

For a given data point  $x^{\mathrm{eval}} $, we can derive the following formula for log likelihood:

$$
\log \rho_t(x^{\mathrm{eval}}) 
= \int_0^1 \frac{t}{1 - t} 
\, \mathbb{E}\!\left[ 
\| \dot{X}_t^{\mathrm{eval}} \|^2 
- 
\| \dot{X}_t^{\mathrm{eval}} - v_t(X_t^{\mathrm{eval}}) \|^2 
\right] \mathrm dt 
\tag{*}
$$

where 
$$
X_t^{\mathrm{eval}} = t x^{\mathrm{eval}} + (1 - t) X_0,
\quad \text{with } X_0 \sim \mathrm{Normal}(0, I).
$$


In [ ]:
x_1 = details["states_used"][-1]

print(x_1.shape)

In [ ]:
@torch.no_grad()
def estimate_loglik_from_KL_chunked(
    v_func,
    x_eval: torch.Tensor,
    n_t: int = 64,
    n_x0: int = 256,
    t_eps: float = 1e-3,
    x0_chunk: int = 64,
    acc_dtype: torch.dtype = torch.float32,
):
    assert 0.0 < t_eps < 0.5
    device, fwd_dtype = x_eval.device, x_eval.dtype
    B = x_eval.shape[0]
    rest_shape = x_eval.shape[1:]
    D = int(torch.tensor(rest_shape).prod().item()) if len(rest_shape) else 1

    X0_full = torch.randn((B, n_x0, *rest_shape), device=device, dtype=fwd_dtype)

    grid = torch.arange(n_t, device=device, dtype=acc_dtype)
    t_grid = (grid + 0.5) / n_t
    t_grid = t_eps + (1.0 - 2.0 * t_eps) * t_grid
    delta_t = (1.0 - 2.0 * t_eps) / n_t
    t_weights = (t_grid / (1.0 - t_grid)) * delta_t  # (n_t,)

    out = torch.zeros((B,), device=device, dtype=acc_dtype)
    x_eval_u1_fwd = x_eval.unsqueeze(1)  # (B,1,...)

    for t_acc, w in tqdm(zip(t_grid, t_weights), total=n_t):
        t = t_acc.to(fwd_dtype)
        w = w.to(acc_dtype)

        accum_b = torch.zeros((B,), device=device, dtype=acc_dtype)

        for i in range(0, n_x0, x0_chunk):
            csz = min(x0_chunk, n_x0 - i)

            X0_chunk = X0_full[:, i:i+csz]  # fwd_dtype
            x_t_chunk = t * x_eval_u1_fwd + (1.0 - t) * X0_chunk

            a_flat = (
                x_eval_u1_fwd.to(acc_dtype).expand(-1, csz, *rest_shape)
                - X0_chunk.to(acc_dtype)
            ).reshape(B * csz, -1)  # (B*csz, D)

            x_t_flat = x_t_chunk.reshape(B * csz, *rest_shape)  # fwd_dtype
            t_vec = torch.full((B * csz,), float(t.item()), device=device, dtype=fwd_dtype)
            v = v_func(x_t_flat, t_vec)

            v_flat = v.reshape(B * csz, -1).to(acc_dtype)

            contrib = 2.0 * (a_flat * v_flat).sum(dim=1) - (v_flat * v_flat).sum(dim=1)
            contrib = contrib.view(B, csz).sum(dim=1)

            accum_b += contrib

        out += w * (accum_b / float(n_x0))

    return out

result = estimate_loglik_from_KL_chunked(
    v_func=flow_model,
    x_eval=x_1,
    n_t=50,
    n_x0=128,
    t_eps=1e-3,
)

In [ ]:
plot_images_with_logrho(
    images=details["states_used"][-1],
    log_rho=result,
    nrow=10,
    title="Samples with log ρ",
    digits=2,
    dpi=300,
    fontsize=8,
    group_titles=["ODE vanilla", "ODE inv", "SDE inv", "Test set inv"],
    group_size=n_samples,
    show_group_mean=True,
)